In [155]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from random import randint
from numpy import array, argmax, asarray, zeros
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
import spacy
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
nlp=spacy.load("en")
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import LSTM, Embedding,GRU
from keras.models import Sequential
from keras.layers import Flatten, Masking
from keras.layers.recurrent import SimpleRNN, LSTM

## define function

In [315]:
lem = WordNetLemmatizer()
def lem_sentences(sentence):
    tokens = nltk.word_tokenize(sentence)
    lemmed_tokens = [lem.lemmatize(token) for token in tokens]
    return ' '.join(lemmed_tokens)

def integer_encode_documents(docs,tokenizer):
    return tokenizer.texts_to_sequences(docs)

## data preparation

In [317]:
tag_data = pd.read_csv('tagged_product_attributes.csv')
full_data = pd.read_csv('full_data.csv')


In [318]:
## pick all of tagged 'embellishment' data 
tag_data['attribute_value']=tag_data['attribute_value'].str.lower()
embellishment=tag_data[tag_data['attribute_name'].str.lower()=='embellishment']

In [319]:
## drop duplicate
embellishment_1=embellishment.drop_duplicates(subset=['product_id','attribute_value'])
embellishment_2=pd.concat([embellishment_1,pd.get_dummies(embellishment_1['attribute_value'])],axis=1)
#embellishment_2['attribute_value'].value_counts()
embellishments=list(embellishment_2['attribute_value'].value_counts().index)


In [320]:
#prepare trained data
full=full_data[['product_id','brand','brand_category','product_full_name','description','details']]\
.drop_duplicates(subset=['product_id'])
full=full.fillna('')
full['input']=full[['brand','brand_category','product_full_name','description','details']].agg(' '.join, axis=1).str.lower()
full['input'] = full['input'].replace(r"[^a-z0-9\s]+", "")
embellishment_tag=embellishment_2.merge(full,how='left',on='product_id')
embellishment_tag=embellishment_tag.groupby(['product_id','input'])[embellishments].sum().reset_index()
embellishment_tag['input']=embellishment_tag['input'].apply(lambda x: lem_sentences(x))
embellishment_tag.head()


,product_id,input,ruffles,studs,embroidery,trim,buckles,beaded,lace,mesh,fringe,sequins,rhinestone,crystals,patches,epaulets
0,01DPCG1C1P0MQAV9NMS3N1TDAA,j.crew unknown collection fluted sheath dress ...,0,0,1,0,0,0,0,1,0,0,0,0,0,0
1,01DPD1R0F6Z4EJCPNDKPAG5DPF,veronica beard unknown lebone pant our lebone ...,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,01DPD2FGA0FWFXJCKMT5QBJYHK,veronica beard unknown ravello skirt in our pl...,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,01DPD3AGTNRNAKNP8TH9WYG35X,veronica beard jacket theron jacket our theron...,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,01DPD4R5X5TQCWTVTC2AEAFC10,veronica beard dress ida dress our ida floral ...,1,0,0,0,0,0,0,0,0,0,0,0,0,0


## Load in GloVe Vectors

In [321]:
def load_glove_vectors():
    embeddings_index = {}
    with open('glove.6B.100d.txt') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index


embeddings_index = load_glove_vectors()

Loaded 400000 word vectors.


##  Define and Fit Model

In [349]:
#try ruffles

inputdata=embellishment_tag['input'].tolist()
label=np.array(embellishment_tag['ruffles'].tolist())
stopwords_removed_docs = list(
    map(lambda doc: " ".join([token.text for token in nlp(doc) if not token.is_stop]), inputdata))
tokenizer.fit_on_texts(stopwords_removed_docs)
encoded_docs = integer_encode_documents(stopwords_removed_docs, tokenizer)
padded_docs = pad_sequences(encoded_docs,padding='post')
labels = to_categorical(encoder.fit_transform(label))
#padded_docs


In [350]:
X_train, X_test, y_train, y_test = train_test_split(padded_docs, labels, test_size=0.2,random_state=0)

In [351]:
VOCAB_SIZE = int(len(tokenizer.word_index) * 1.1)
VOCAB_SIZE 

32412

In [352]:
embedding_matrix = zeros((VOCAB_SIZE, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

In [353]:
def make_binary_classification_rnn_model():
    model = Sequential()
    model.add(Embedding(VOCAB_SIZE, 100, weights=[embedding_matrix],trainable=False))
    model.add(Masking(mask_value=0.0)) 
    model.add(SimpleRNN(units=64))
    model.add(Dense(16))
    model.add(Dense(2, activation='softmax'))
    model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    #model.summary()
    return model

In [354]:
model = make_binary_classification_rnn_model()


In [355]:
history = model.fit(X_train, y_train,validation_split = 0.1, epochs=5, verbose=1)

Train on 236 samples, validate on 27 samples
Epoch 1/5
236/236 [==============================] - 1s 5ms/step - loss: 0.6117 - accuracy: 0.7161 - val_loss: 0.5357 - val_accuracy: 0.7778
Epoch 2/5
236/236 [==============================] - 1s 2ms/step - loss: 0.4769 - accuracy: 0.7712 - val_loss: 0.4648 - val_accuracy: 0.8148
Epoch 3/5
236/236 [==============================] - 1s 3ms/step - loss: 0.3861 - accuracy: 0.8305 - val_loss: 0.4558 - val_accuracy: 0.8889
Epoch 4/5
236/236 [==============================] - 1s 2ms/step - loss: 0.3434 - accuracy: 0.8517 - val_loss: 0.4166 - val_accuracy: 0.8889
Epoch 5/5
236/236 [==============================] - 1s 2ms/step - loss: 0.2965 - accuracy: 0.8814 - val_loss: 0.3998 - val_accuracy: 0.9259


## evaluate every tag's RNN performance

In [358]:
import random
random.seed(0)
def value_model(i):
    inputdata=embellishment_tag['input'].tolist()
    label=np.array(embellishment_tag["{}".format(i)].tolist())
    stopwords_removed_docs = list(
        map(lambda doc: " ".join([token.text for token in nlp(doc) if not token.is_stop]), inputdata))
    tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
    tokenizer.fit_on_texts(stopwords_removed_docs)
    encoded_docs = integer_encode_documents(stopwords_removed_docs, tokenizer)
    padded_docs = pad_sequences(encoded_docs, padding='post')
    labels = to_categorical(encoder.fit_transform(label))
    X_train, X_test, y_train, y_test = train_test_split(padded_docs, labels, test_size=0.2,random_state=0)
    VOCAB_SIZE = int(len(tokenizer.word_index) * 1.1)
    embedding_matrix = zeros((VOCAB_SIZE, 100))
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
    model = make_binary_classification_rnn_model()
    return model.fit(X_train, y_train,validation_split = 0.1, epochs=5, verbose=1)

In [359]:
for i in embellishments:
    print(f'{i} prediction performance:')
    value_model(i)

ruffles prediction performance:
Train on 236 samples, validate on 27 samples
Epoch 1/5
236/236 [==============================] - 1s 5ms/step - loss: 0.6509 - accuracy: 0.6398 - val_loss: 0.5575 - val_accuracy: 0.7778
Epoch 2/5
236/236 [==============================] - 1s 2ms/step - loss: 0.4955 - accuracy: 0.7839 - val_loss: 0.5290 - val_accuracy: 0.8148
Epoch 3/5
236/236 [==============================] - 1s 2ms/step - loss: 0.4289 - accuracy: 0.8178 - val_loss: 0.5529 - val_accuracy: 0.8148
Epoch 4/5
236/236 [==============================] - 1s 2ms/step - loss: 0.3617 - accuracy: 0.8517 - val_loss: 0.4994 - val_accuracy: 0.8148
Epoch 5/5
236/236 [==============================] - 1s 2ms/step - loss: 0.3364 - accuracy: 0.8686 - val_loss: 0.5002 - val_accuracy: 0.8148
studs prediction performance:
Train on 236 samples, validate on 27 samples
Epoch 1/5
236/236 [==============================] - 1s 6ms/step - loss: 0.5965 - accuracy: 0.6949 - val_loss: 0.4811 - val_accuracy: 0.8148
Ep

## Set model for every tag

In [360]:
# In the following section, our team fit a model for each tag to make a binary prediction whether the record have that type of embellishment
def tag_model(i):
    inputdata=embellishment_tag['input'].tolist()
    label=np.array(embellishment_tag["{}".format(i)].tolist())
    stopwords_removed_docs = list(
        map(lambda doc: " ".join([token.text for token in nlp(doc) if not token.is_stop]), inputdata))
    tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
    tokenizer.fit_on_texts(stopwords_removed_docs)
    encoded_docs = integer_encode_documents(stopwords_removed_docs, tokenizer)
    padded_docs = pad_sequences(encoded_docs, padding='post')
    labels = to_categorical(encoder.fit_transform(label))
    X_train, X_test, y_train, y_test = train_test_split(padded_docs, labels, test_size=0.2,random_state=0)
    VOCAB_SIZE = int(len(tokenizer.word_index) * 1.1)
    embedding_matrix = zeros((VOCAB_SIZE, 100))
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
    model = make_binary_classification_rnn_model()
    model.fit(X_train, y_train,validation_split = 0.1, epochs=5, verbose=1)
    return model


In [361]:
#store all of the trained models in a series for future use
tag_models=pd.Series([tag_model('{}'.format(i)) for i in embellishments],index=embellishments)


Train on 236 samples, validate on 27 samples
Epoch 1/5
236/236 [==============================] - 1s 6ms/step - loss: 0.7086 - accuracy: 0.6059 - val_loss: 0.6363 - val_accuracy: 0.7407
Epoch 2/5
236/236 [==============================] - 1s 2ms/step - loss: 0.4923 - accuracy: 0.7966 - val_loss: 0.6368 - val_accuracy: 0.7778
Epoch 3/5
236/236 [==============================] - 1s 2ms/step - loss: 0.3989 - accuracy: 0.8263 - val_loss: 0.5979 - val_accuracy: 0.7778
Epoch 4/5
236/236 [==============================] - 1s 3ms/step - loss: 0.3377 - accuracy: 0.8602 - val_loss: 0.5903 - val_accuracy: 0.7778
Epoch 5/5
236/236 [==============================] - 1s 3ms/step - loss: 0.2965 - accuracy: 0.8686 - val_loss: 0.6152 - val_accuracy: 0.8148
Train on 236 samples, validate on 27 samples
Epoch 1/5
236/236 [==============================] - 1s 6ms/step - loss: 0.7545 - accuracy: 0.6271 - val_loss: 0.5226 - val_accuracy: 0.7778
Epoch 2/5
236/236 [==============================] - 1s 2ms/step

## Make Prediction

In [377]:
#define tokenizer
inputdata=embellishment_tag['input'].tolist()
stopwords_removed_docs = list(
map(lambda doc: " ".join([token.text for token in nlp(doc) if not token.is_stop]), inputdata))
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(stopwords_removed_docs)

In [378]:
description=""##input 
brand=""
brand_category=""
product_full_name=""
details=""
def Identify_embellishment_tag(description, brand, brand_category, product_full_name,details):
    inputdata=[brand,brand_category,product_full_name,description,details]
    inputdata=''.join(inputdata).lower()
    inputdata=inputdata.replace(r"[^a-z0-9\s]+", "")
    inputdata=lem_sentences(inputdata)
    inputdata=[inputdata]
    test_docs = list(
        map(lambda doc: " ".join([token.text for token in nlp(doc) if not token.is_stop]), inputdata))
    encoded_docs = integer_encode_documents(test_docs, tokenizer)
    padded_docs = pad_sequences(encoded_docs, padding='post')
    tags=[]
    for i in embellishments:
        if tag_models[i].predict_classes(padded_docs)==1:
            tags.append(i)
    tags=" ".join(tags)
    print(f'predicted embellishment: {tags}')
    
    
    

In [379]:
#try to test using the row for what we have already known the data
testrow=full.loc[full['product_id']=='01DPCG1C1P0MQAV9NMS3N1TDAA',:]
brand="J.Crew"
brand_category="Unknown"
product_full_name="Collection fluted sheath dress in Ratti® climb..."
details=""
Identify_embellishment_tag(description, brand, brand_category, product_full_name,details)## test is good

predicted embellishment: trim


In [380]:
# make prediction on the full data set
Full_data = pd.read_csv('full_data.csv')
Full_data.head()

,product_id,brand,mpn,product_full_name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,details,labels,bc_product_id
0,01DSE9TC2DQXDG6GWKW9NMJ416,Banana Republic,514683,Ankle-Strap Pump,"A modern pump, in a rounded silhouette with an...",Unknown,2019-11-11 22:37:15.719107+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,"A modern pump, in a rounded silhouette with an...","{""Needs Review""}",NaN
1,01DSE9SKM19XNA6SJP36JZC065,Banana Republic,526676,Petite Tie-Neck Top,Dress it down with jeans and sneakers or dress...,Unknown,2019-11-11 22:36:50.682513+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,Dress it down with jeans and sneakers or dress...,"{""Needs Review""}",NaN
2,01DSJX8GD4DSAP76SPR85HRCMN,Loewe,4.001E+11,52MM Padded Leather Round Sunglasses,Padded leather covers classic round sunglasses.,JewelryAccessories/SunglassesReaders/RoundOval...,2019-11-13 17:33:59.581661+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/loewe-52mm-pad...,100% UV protection\nCase and cleaning cloth in...,"{""Needs Review""}",NaN
3,01DSJVKJNS6F4KQ1QM6YYK9AW2,Converse,4.00012E+11,Baby's & Little Kid's All-Star Two-Tone Mid-To...,The iconic mid-top design gets an added dose o...,"JustKids/Shoes/Baby024Months/BabyGirl,JustKids...",2019-11-13 17:05:05.203733+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/converse-babys...,Canvas upper\nRound toe\nLace-up vamp\nSmartFO...,"{""Needs Review""}",NaN
4,01DSK15ZD4D5A0QXA8NSD25YXE,Alexander McQueen,4.00011E+11,64MM Rimless Sunglasses,Hexagonal shades offer a rimless view with int...,JewelryAccessories/SunglassesReaders/RoundOval,2019-11-13 18:42:30.941321+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/alexander-mcqu...,100% UV protection\nGradient lenses\nAdjustabl...,"{""Needs Review""}",NaN


In [381]:
# In the following section, we designed a function that will take input for an item, implement data preprocessing process,
# and record the prediction result generated by models of different tags
def full_embellishment_tag(description, brand, brand_category, product_full_name,details):
    inputdata=[brand,brand_category,product_full_name,description,details]
    inputdata=''.join(inputdata).lower()
    inputdata=inputdata.replace(r"[^a-z0-9\s]+", "")
    inputdata=lem_sentences(inputdata)
    inputdata=[inputdata]
    test_docs = list(
        map(lambda doc: " ".join([token.text for token in nlp(doc) if not token.is_stop]), inputdata))
    encoded_docs = integer_encode_documents(test_docs, tokenizer)
    padded_docs = pad_sequences(encoded_docs, padding='post')
    tags=[]
    for i in embellishments:
        if tag_models[i].predict_classes(padded_docs)==1:
            tags.append(i)
    tags=" ".join(tags)
    return tags

In [ ]:
embellishment_output = []
Full_data=Full_data.fillna('')
inputdata = Full_data

for i in range(len(inputdata)):
    description = inputdata.loc[i,"description"]
    brand = inputdata.loc[i,"brand"] 
    brand_category = inputdata.loc[i,"brand_category"] 
    product_full_name = inputdata.loc[i,"product_full_name"] 
    details = inputdata.loc[i,"details"] 
    tags=full_embellishment_tag(description, brand, brand_category, product_full_name,details)
    print(f'{i}:{tags}')
    embellishment_output.append(tags)

In [ ]:
embellishment_output

In [386]:
Full_data['embellishment']=pd.Series(embellishment_output)

In [388]:
Full_data['embellishment'].value_counts()

                                                  37743
ruffles                                            4448
studs                                              1578
trim                                               1058
embroidery                                          788
buckles                                             591
beaded                                              573
fringe                                              188
mesh                                                150
sequins                                             116
lace                                                113
ruffles studs                                       110
ruffles embroidery                                   98
ruffles beaded                                       90
epaulets                                             89
ruffles buckles                                      79
ruffles trim                                         74
studs trim                                      

In [389]:
Full_data.to_csv('full data with embellishment.csv')

In [391]:
pd.read_csv('full data with embellishment.csv')

,Unnamed: 0,product_id,brand,mpn,product_full_name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,details,labels,bc_product_id,embellishment
0,0,01DSE9TC2DQXDG6GWKW9NMJ416,Banana Republic,514683,Ankle-Strap Pump,"A modern pump, in a rounded silhouette with an...",Unknown,2019-11-11 22:37:15.719107+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,"A modern pump, in a rounded silhouette with an...","{""Needs Review""}",NaN,NaN
1,1,01DSE9SKM19XNA6SJP36JZC065,Banana Republic,526676,Petite Tie-Neck Top,Dress it down with jeans and sneakers or dress...,Unknown,2019-11-11 22:36:50.682513+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,Dress it down with jeans and sneakers or dress...,"{""Needs Review""}",NaN,NaN
2,2,01DSJX8GD4DSAP76SPR85HRCMN,Loewe,4.001E+11,52MM Padded Leather Round Sunglasses,Padded leather covers classic round sunglasses.,JewelryAccessories/SunglassesReaders/RoundOval...,2019-11-13 17:33:59.581661+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/loewe-52mm-pad...,100% UV protection\nCase and cleaning cloth in...,"{""Needs Review""}",NaN,NaN
3,3,01DSJVKJNS6F4KQ1QM6YYK9AW2,Converse,4.00012E+11,Baby's & Little Kid's All-Star Two-Tone Mid-To...,The iconic mid-top design gets an added dose o...,"JustKids/Shoes/Baby024Months/BabyGirl,JustKids...",2019-11-13 17:05:05.203733+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/converse-babys...,Canvas upper\nRound toe\nLace-up vamp\nSmartFO...,"{""Needs Review""}",NaN,NaN
4,4,01DSK15ZD4D5A0QXA8NSD25YXE,Alexander McQueen,4.00011E+11,64MM Rimless Sunglasses,Hexagonal shades offer a rimless view with int...,JewelryAccessories/SunglassesReaders/RoundOval,2019-11-13 18:42:30.941321+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/alexander-mcqu...,100% UV protection\nGradient lenses\nAdjustabl...,"{""Needs Review""}",NaN,NaN
5,5,01DSRCS6FWFEXE6T3QYZ7QHN7V,Kissy Kissy,4.00098E+11,Baby Girl's Long-Sleeve Cotton Footie,Comfortable cotton footie with scalloped trim,JustKids/Baby024months/InfantGirls/FootiesRompers,2019-11-15 20:41:27.284292+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/kissy-kissy-ba...,Crewneck\nLong sleeves\nFront snap closure\nPi...,"{""Needs Review""}",NaN,NaN
6,6,01DSE9J9B1WCDKEZDBXY9JCD1H,Banana Republic,512628,Untucked Slim-Fit Tech-Stretch Cotton Shirt,"Made with our Tech Stretch Cotton fabric, this...",Unknown,2019-11-11 22:32:50.778391+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,"Made with our Tech Stretch Cotton fabric, this...","{""Needs Review""}",NaN,NaN
7,7,01DSEAZS5R8HJBGWY3XHJSDC1W,Banana Republic,402448,Classic-Fit Bi-Stretch Blazer,Our Classic Fit blazer has a single-button fro...,Unknown,2019-11-11 22:57:41.549451+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,Our Classic Fit blazer has a single-button fro...,"{""Needs Review""}",NaN,NaN
8,8,01DSE9HTZN1C0YDB1W421K6TY0,Banana Republic,517546,Print Puff-Sleeve Wrap Dress,Add a little feminine flair to your look with ...,Unknown,2019-11-11 22:32:36.077057+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,Add a little feminine flair to your look with ...,"{""Needs Review""}",NaN,NaN
9,9,01DSJT9E7E6K26W1F0CP4H2E8H,Hunter,4.00011E+11,Hunter x Peppa Pig Print Umbrella,From the Hunter x Peppa Pig Collaboration. Lig...,"JustKids/Girls214/Accessories,JustKids/Boys220...",2019-11-13 16:42:04.392452+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/hunter-hunter-...,Easy-to-hold curved handle\nSafety feature on ...,"{""Needs Review""}",NaN,NaN
